# Interrupt

This page considers the ways to implement a temporary interruption to the graph, a typical solution for different human-in-the-loop use cases.

In [2]:
from typing import TypedDict

import langgraph
from langgraph.types import interrupt
from langgraph.graph import StateGraph, START, END

## Continue execution

The main purpose of the `interrupt` is `langgraph` is to build human in the loop applications: in which user can provide additional information to the LLM based application during the application execution.

To do that you have too:

- Add memory to your graph.
- The `interrupt` invocation returns to the graph information provided from the flow outside the graph.
- To resume the execution, invoke graph again with `langgraph.types.Command` providing additional information via the `resume` argument.

---

The following cell defines the graph in which a single node calls `interupt` and publishes the result to the `State`.

In [3]:
from langgraph.checkpoint.memory import InMemorySaver


class State(TypedDict):
    message_from_user: str | None


def node(state: State) -> State:
    ans = interrupt("Expecting input")
    return State(message_from_user=ans)


graph = (
    StateGraph(State)
    .add_node("a", node)
    .add_edge(START, "a")
    .add_edge("a", END)
    .compile(checkpointer=InMemorySaver())
)

The `config` of the graph invocation is important here as it specifies which thread was interrupted.

In [4]:
config = {"configurable": {"thread_id": 1}}

ans = graph.invoke(
    State(message_from_user=None),
    config=config
)
ans

{'message_from_user': None,
 '__interrupt__': [Interrupt(value='Expecting input', id='c40e73e4bf24e1f3e6088f0f02ac16e4')]}

The graph is interrupted. The next cell shows the how to resume the execution and provide the information to the graph.

In [5]:
graph.invoke(
    langgraph.types.Command(resume="This is input from user"),
    config=config
)

{'message_from_user': 'This is input from user'}